In [1]:
import numpy as np
import matplotlib.pyplot as plt

import misc_data_utils

In [2]:
data = misc_data_utils.get_snp_data(start="2000-01-01")

[*********************100%***********************]  505 of 505 completed


In [3]:
raw_data = data["Adj Close"].pct_change().values[1:]

In [4]:
isnan_bool = np.isnan(raw_data[0])
nan_cols = np.where(isnan_bool)[0]

raw_data_clean = raw_data[:, np.where(1 - isnan_bool)[0]]

In [7]:
nan_data = raw_data[:, nan_cols]
rows, cols = np.where(np.isnan(nan_data))

cleaned_data = []
for i in range(nan_data.shape[1]):
    last_nan_row = np.max(rows[np.where(cols == i)])
    cleaned_data.append(nan_data[:, i][last_nan_row + 1:])

for i in range(raw_data_clean.shape[1]):
    cleaned_data.append(raw_data_clean[:, i])

In [ ]:
np.isnan(cleaned_data[i]) for i in range(len(cleaned_data))]